In [2]:
# IMPORTING LIBRARIES

import pandas as pd                                     
import os                                               
from scipy.stats import gaussian_kde as kde
import random

%matplotlib inline
import seaborn as sns                                   # For pretty plots

In [5]:
# Import original data, then do a bit of data-munging to get it in displayable form.
filename='CrowdstormingDataJuly1st.csv'
original_df = pd.read_csv(filename)

In [23]:
original_df.shape

(146028, 28)

In [6]:
original_df.describe()

//anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


height         weight          games      victories  \
count  145765.000000  143785.000000  146028.000000  146028.000000   
mean      181.935938      76.075662       2.921166       1.278344   
std         6.738726       7.140906       3.413633       1.790725   
min       161.000000      54.000000       1.000000       0.000000   
25%              NaN            NaN       1.000000       0.000000   
50%              NaN            NaN       2.000000       1.000000   
75%              NaN            NaN       3.000000       2.000000   
max       203.000000     100.000000      47.000000      29.000000   

                ties        defeats          goals    yellowCards  \
count  146028.000000  146028.000000  146028.000000  146028.000000   
mean        0.708241       0.934581       0.338058       0.385364   
std         1.116793       1.383059       0.906481       0.795333   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       1.000000       0.000000       0.000000   
75%         1.000000       1.000000       0.000000       1.000000   
max        14.000000      18.000000      23.000000      14.000000   

          yellowReds       redCards         rater1         rater2  \
count  146028.000000  146028.000000  124621.000000  124621.000000   
mean        0.011381       0.012559       0.264255       0.302862   
std         0.107931       0.112889       0.295382       0.293020   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000            NaN            NaN   
50%         0.000000       0.000000            NaN            NaN   
75%         0.000000       0.000000            NaN            NaN   
max         3.000000       2.000000       1.000000       1.000000   

              refNum     refCountry        meanIAT          nIAT  \
count  146028.000000  146028.000000  145865.000000  1.458650e+05   
mean     1534.827444      29.642842       0.346276  1.969741e+04   
std       918.736625      27.496189       0.032246  1.271262e+05   
min         1.000000       1.000000      -0.047254  2.000000e+00   
25%       641.000000       7.000000            NaN           NaN   
50%      1604.000000      21.000000            NaN           NaN   
75%      2345.000000      44.000000            NaN           NaN   
max      3147.000000     161.000000       0.573793  1.975803e+06   

              seIAT        meanExp          nExp          seExp  
count  1.458650e+05  145865.000000  1.458650e+05  145865.000000  
mean   6.310849e-04       0.452026  2.044023e+04       0.002994  
std    4.735857e-03       0.217469  1.306157e+05       0.019723  
min    2.235373e-07      -1.375000  2.000000e+00       0.000001  
25%             NaN            NaN           NaN            NaN  
50%             NaN            NaN           NaN            NaN  
75%             NaN            NaN           NaN            NaN  
max    2.862871e-01       1.800000  2.029548e+06       1.060660

In [7]:
original_df.head()

playerShort         player             club leagueCountry    birthday  \
0  lucas-wilchez  Lucas Wilchez    Real Zaragoza         Spain  31.08.1983   
1     john-utaka     John Utaka  Montpellier HSC        France  08.01.1982   
2    abdon-prats    Abdón Prats     RCD Mallorca         Spain  17.12.1992   
3     pablo-mari     Pablo Marí     RCD Mallorca         Spain  31.08.1993   
4     ruben-pena     Rubén Peña  Real Valladolid         Spain  18.07.1991   

   height  weight              position  games  victories    ...     rater2  \
0   177.0    72.0  Attacking Midfielder      1          0    ...       0.50   
1   179.0    82.0          Right Winger      1          0    ...       0.75   
2   181.0    79.0                   NaN      1          0    ...        NaN   
3   191.0    87.0           Center Back      1          1    ...        NaN   
4   172.0    70.0      Right Midfielder      1          1    ...        NaN   

   refNum  refCountry  Alpha_3   meanIAT    nIAT     seIAT   meanExp    nExp  \
0       1           1      GRC  0.326391   712.0  0.000564  0.396000   750.0   
1       2           2      ZMB  0.203375    40.0  0.010875 -0.204082    49.0   
2       3           3      ESP  0.369894  1785.0  0.000229  0.588297  1897.0   
3       3           3      ESP  0.369894  1785.0  0.000229  0.588297  1897.0   
4       3           3      ESP  0.369894  1785.0  0.000229  0.588297  1897.0   

      seExp  
0  0.002696  
1  0.061504  
2  0.001002  
3  0.001002  
4  0.001002  

[5 rows x 28 columns]

### Disaggregating

In [26]:
grouped = original_df[['refNum', 'games']].groupby(['refNum']).sum()
grouped_df = grouped[grouped['games'] >= 22].reset_index()
grouped_df.shape

(1261, 2)

In [28]:
grouped_df['refNum'].values

array([   4,    7,   15, ..., 3139, 3144, 3147])

In [29]:
referees_df = original_df[original_df['refNum'].isin(grouped_df['refNum'].values)]
referees_df.shape

(137635, 28)

In [32]:
games = referees_df['games'].sum()
games

415692

### TODOs:
    - sklearn.ensemble.RandomForestClassifier to create a colour skin prediction models
    - Find out what is the baseline truth on which our ML Model should be tested (try it out with mean of rater 1 & 2)